# Loading Required dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

import os

# Function to load data

All the data are saved in the `data` folder. It contains not only the raw data, but also all the preprocessed data since I donot want to have to keep running the prepocessing steps. So I would like to create a function that loads all the data from the subfolders in the data folder as needed.

In [6]:
def get_data(folder_name, file_name):
    """
        loads the 'file_name' csv file from the 'folder_name' folder
        get_data(folder_name, file_name) : Str Str -> Pandas DataFrame 
    """
    source_path = os.path.join('..\data', folder_name)
    if file_name == 'sample_submission.csv':
        path_data = os.path.join(source_path, file_name)
        data_df = pd.read_csv(path_data)
    elif file_name == 'train.csv':
        path_data = os.path.join(source_path, file_name)
        data_df = pd.read_csv(path_data)
    elif file_name == 'games.csv':
        path_data = os.path.join(source_path, file_name)
        data_df = pd.read_csv(path_data, parse_dates = ['created_at'])
    elif file_name == 'turns.csv':
        path_data = os.path.join(source_path, file_name)
        data_df = pd.read_csv(path_data)
    else:
        path_data = os.path.join(source_path, file_name)
        data_df = pd.read_csv(path_data)
    return data_df

## Test
train_data = get_data('main_data', 'train.csv')
train_data.head()

,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


# Creating a custom Scikit-Learn Transformer:

we need to predict the ratings for the human players but we are given data for both human and bot players for each game. So, the data for the bot players will clearly be helpful in predicting the human players ratings, so I would like to extract the data for the bots and add them as feature columns for the human players

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class AddBotFeatures (BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    def fit(self, X):
        return self
    def transform (self, X):
        bot_names = ['BetterBot', 'STEEBot', 'HastyBot']
        bot_data = X.loc[X['nickname'].isin(bot_names)].copy()
        bot_data.rename(columns={'nickname':'bot_nickname', 'score': 'bot_score', 'rating': 'bot_rating'}, inplace= True )
        human_data = X.loc[~X['nickname'].isin(bot_names)].copy()
        
        # Join the two dataframe
        new_df = human_data.join(bot_data.set_index('game_id'), on='game_id')

        # Move the rating column to the end
        column_to_move = new_df.pop("rating")

        new_df.insert(6, "rating", column_to_move)

        return new_df


# Test
new_train = AddBotFeatures()

training_data = new_train.fit_transform(train_data)

training_data.head()
        

,game_id,nickname,score,bot_nickname,bot_score,bot_rating,rating
1,1,stevy,429,BetterBot,335,1637,1500
2,3,davidavid,440,BetterBot,318,2071,1811
4,4,Inandoutworker,119,BetterBot,478,1936,1473
6,5,stevy,325,STEEBot,427,1844,1500
8,6,HivinD,378,STEEBot,427,2143,2029


# Split data into train-test-val

In [21]:
from sklearn.model_selection import train_test_split


def split_data(df):
    """Splits df into training, testing and validation sets
        split_data: Pandas DataFrame -> Pandas DataFrame, Pandas DataFrame, Pandas DataFrame, Pandas DataFrame
    """
    X_data = df.drop(columns = "rating")
    train_y = df["rating"].copy()
    X_train, X_test, y_train, y_test = train_test_split(X_data, train_y, test_size=0.3, random_state=123)
    return X_train, X_test, y_train, y_test

# Test
X_train, X_test, y_train, y_test = split_data(training_data)

X_test.shape                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

(15123, 6)

In [22]:
X_train.head(5)

,game_id,nickname,score,bot_nickname,bot_score,bot_rating
38284,27623,BB-8,440,HastyBot,333,1664
16334,11732,jodel,380,BetterBot,425,1966
64835,46774,BB-8,430,HastyBot,307,1640
61653,44462,Yuki86,260,STEEBot,630,2071
41920,30222,PVMG,395,BetterBot,364,1810
